# Scraping

In this first step, we will capture the data from the internet. The idea is to create a scraper to gather data regarding house pricing. Now, there are a lot of pages where we could obtain that data. To this specific exercise, we will use the folloging web pages;

 * https://casas.trovit.com.co/arriendo-valle-aburra
 * https://www.espaciourbano.com/listado_arriendos.asp?sZona=1
 * https://inmobiliariasantillana.co/propiedades/?filter=true&tipo_de_negocio=Arriendo
 * https://www.fincaraiz.com.co/arriendo?&searchstring=bello
 * https://www.fincaraiz.com.co/arriendo/casas-y-apartamentos-y-edificios-y-apartaestudios-y-cabanas-y-casas-campestres/antioquia
 * https://www.metrocuadrado.com/apartaestudio-apartamento-casa-edificio-de-apartamentos/arriendo/medellin/?search=form
  
NOTA: fincaraiz.com requieres Selenium. metrocuadrado.com as well. 

# First page: casas.trovit.com.co

Let's first create an schema to obtain all data from this source. At the very begginig we will need to import scraping libraries

Vamos a acceder a cada una de las páginas de la web y revisar de donde vamos a sacar cada uno de los datos qe necesitamos.

Entendiendo en  que contenedores se almacena la información, vamos a hacer una extracción de ejmplo para ver como la trae beautifulsoup

a manera de ejemplo, vamos a sacar la información de la página https://casas.trovit.com.co/detail/0194c892-64b0-7797-996b-88bff0e32618?type=direct

vamos a extraer: area, habitaciones, precio, parqueadero, baños, latitud, longitud, estrato, tipo de propiedad, ciudad, departamento, tipo de operación, año de construcción, descripción



In [1]:
# Scraping libraries
import requests
from bs4 import BeautifulSoup
# import urllib


In [5]:

url = "https://casas.trovit.com.co/detail/0194c892-64b0-7797-996b-88bff0e32618?type=direct"
response = requests.get(url)

# Looking into the HTML
html_content = response.text

with open("anuncio.txt", "w", encoding='utf-8') as file:
    file.write(html_content)
    print('HTML Content has been saved succcesfully')

html_content[:100] # Looking into the first 100 characters


HTML Content has been saved succcesfully


'<!doctype html>\n<html lang="es">\n<head>\n    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n'

# Extraqcting General info

In [7]:
# Ahora vamos a buscar la sección <section class="features"> en el BeautifulSoup
features_section = soup.find("section", class_="features")

# Dentro de esta sección, buscamos todos los <li class="main-features__item">
features_items = features_section.find_all("li", class_="main-features__item") if features_section else []

# Extraemos pares clave-valor del contenido de cada <li>
info_dict = {}
for item in features_items:
    key_tag = item.find("div", class_="key")
    value_tag = item.find("div", class_="value")
    if key_tag and value_tag:
        key = key_tag.get_text(strip=True)
        value = value_tag.get_text(strip=True)
        info_dict[key] = value

info_dict


{'Tipo de operación': 'Arriendo',
 'Tipo de propiedad': 'Apartamento',
 'Habitaciones': '2',
 'Baños': '3',
 'Área total': '117 m²',
 'Año de construcción': '2024',
 'Estrato': '6'}

# Extracting Long-Lat and address

In [22]:
import re

# 1. Buscar el script que contiene 'mapData'
script_mapdata = None
for tag in soup.find_all("script"):
    if tag.string and "mapData" in tag.string:
        script_mapdata = tag.string
        break

# 2. Buscar valores específicos con regex
if script_mapdata:
    lat = re.search(r'latitude:\s*"([^"]+)"', script_mapdata)
    lon = re.search(r'longitude:\s*"([^"]+)"', script_mapdata)
    province = re.search(r'province:\s*"([^"]+)"', script_mapdata)
    locality = re.search(r'locality:\s*"([^"]+)"', script_mapdata)
    address = re.search(r'address:\s*`([^`]+)`', script_mapdata)

    map_info = {
        "latitud": lat.group(1) if lat else None,
        "longitud": lon.group(1) if lon else None,
        "departamento": province.group(1) if province else None,
        "ciudad": locality.group(1) if locality else None,
        "dirección": address.group(1) if address else None,
    }
else:
    map_info = "No se encontró mapData en ningún script."

map_info


{'latitud': '6.216726',
 'longitud': '-75.553827',
 'departamento': 'Antioquia',
 'ciudad': 'Medellín',
 'dirección': 'Via Las Palmas Al Lado Derecho De Primavera Kr 22 16 30, Medellín, Antioquia, COL'}